In [ ]:
import cv2
import math
from ultralytics import YOLO

# Constants
FPS = 30  # Use actual FPS of the video if different
PIXELS_PER_METER = 10  # Adjust this based on your video
model = YOLO("yolov8n.pt")

# Load video
video_path = "clip6.mp4"
cap = cv2.VideoCapture(video_path)

# Output video writer
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) or FPS
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_video6.mp4", fourcc, fps, (width, height))

# Vehicle tracking dictionary {id: (prev_center, speed)}
track_data = {}

# Vehicle classes of interest
vehicle_classes = ["car", "motorcycle", "bus", "truck", "auto"]
vehicle_colors = {
    "car": (255, 0, 255),
    "motorcycle": (255, 0, 0),
    "bus": (0, 0, 255),
    "truck": (0, 125, 255),
    #"auto": (255, 125, 0)
}

def draw_label_with_background(img, text, topleft, color, font_scale=1.0, thickness=2):
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_w, text_h = text_size
    x, y = topleft

    # Draw filled rectangle (label background)
    cv2.rectangle(img, (x, y - text_h - 10), (x + text_w+5, y), color, -1)

    # Put text over it
    cv2.putText(img, text, (x, y - 5), font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

def estimate_speed(p1, p2):
    dist = math.hypot(p2[0] - p1[0], p2[1] - p1[1])  # Euclidean distance
    meters = dist / PIXELS_PER_METER
    speed = meters * fps * 3.6  # km/h
    return round(speed, 2)

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Detect objects
    results = model(frame, conf=0.5, verbose=False)[0]

    for i, box in enumerate(results.boxes):
        cls_id = int(box.cls[0])
        class_name = model.names[cls_id]
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        x1, y1, x2, y2 = xyxy
        center = ((x1 + x2) // 2, (y1 + y2) // 2)

        if class_name in vehicle_classes:
            color = vehicle_colors.get(class_name, (255, 255, 255))

            # Estimate speed using center point tracking
            obj_id = f"{class_name}_{i}"  # basic ID for simplicity
            prev_center = track_data.get(obj_id, (center, 0))[0]
            speed = estimate_speed(prev_center, center) if frame_count > 1 else 0
            track_data[obj_id] = (center, speed)

            # Draw rectangle and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            label = f"{class_name} {speed} km/h"
            draw_label_with_background(frame, label, (x1, y1), color)


        elif class_name == "person":
            person_box = box.xyxy[0]
            has_helmet = False

            # Check for helmets in the same frame
            for other in results.boxes:
                other_cls = int(other.cls[0])
                other_name = model.names[other_cls]

                if other_name == "helmet":
                    helmet_box = other.xyxy[0]
                    iou = (
                        max(0, min(person_box[2], helmet_box[2]) - max(person_box[0], helmet_box[0])) *
                        max(0, min(person_box[3], helmet_box[3]) - max(person_box[1], helmet_box[1]))
                    )
                    if iou > 0:
                        has_helmet = True
                        break

            if not has_helmet:
                draw_label_with_background(frame, "no Helmet", (int(person_box[0]), int(person_box[1])), (0, 0, 255))

                cv2.rectangle(frame, (int(person_box[0]), int(person_box[1])),
                              (int(person_box[2]), int(person_box[3])), (0, 0, 255), 2)

    out.write(frame)
    # Optional: show live frame while processing
    # cv2.imshow("Traffic Detection", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Done! Output saved as output_video3.mp4")


✅ Done! Output saved as output_video3.mp4


In [12]:
import streamlit as st
x = open('clip1.mp4','rb')
vi = x.read()
st.video(vi)


2025-04-24 11:10:31.474 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 11:10:31.482 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 11:10:32.133 
  command:

    streamlit run C:\Users\sneha\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-24 11:10:32.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()